# Практика

Скоринг учащихся (следующий семестр на основе текущего, модель с учётом дисциплины)

Команда
- Product owner: Антонов Илья
- Scrum-master: Нейман Алексей
- Team: Лебкова Марина, Чвиков Матвей, Махров Матвей, Савин Алексей, Труфманов Михаил

## Загрузка библиотек

In [1]:
!pip install catboost
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.metrics import f1_score, mean_squared_error, mean_absolute_error, r2_score
from catboost import Pool, CatBoost, CatBoostRegressor, cv

## Предобработка данных

In [3]:
pre_data = pd.read_excel('Успеваемость_01.xlsx')

In [4]:
pre_data.to_csv('/content/new_data.csv', index=False)

In [69]:
df_new = pd.read_csv('new_data.csv')

In [70]:
df_new.head(1)

,hash,Номер ЛД,Уровень подготовки,Учебная группа,Специальность/направление,Учебный год,Полугодие,Дисциплина,Оценка (без пересдач),Оценка (успеваемость)
0,67a80fffd8d0294a596eda117d7e393c,2010218,Академический бакалавр,БИВТ-20-4,Информатика и вычислительная техника,2020 - 2021,I полугодие,Инженерная компьютерная графика,Хорошо,Хорошо


In [71]:
df_new = df_new.replace({'I полугодие': 1, 'II полугодие': 2})

#### Удаление данных

Удаление учебных годов (практически) без оценок

In [72]:
years_to_exclude = ['2023 - 2024', '2024 - 2025', '2025 - 2026', '2026 - 2027', '2027 - 2028']

df_new = df_new[~df_new['Учебный год'].isin(years_to_exclude)].copy()

In [73]:
df_new = df_new[~df_new['Учебная группа'].str.contains('22')].copy()

Удаление предметов без оценки

In [74]:
df_new = df_new.dropna(subset=['Оценка (без пересдач)', 'Оценка (успеваемость)'], how='all')

#### Заполнение пропусков в промежуточной аттестации

при условии, что есть итоговая оценка

In [75]:
df_new['Оценка (без пересдач)'] = df_new.apply(
    lambda row: 2 if pd.isna(row['Оценка (без пересдач)']) and not pd.isna(row['Оценка (успеваемость)']) else row['Оценка (без пересдач)'],
    axis=1
)

#### Добавление столбца с семестром

In [76]:
def calculate_semester(row):
    group_year = int(row['Учебная группа'].split('-')[1])
    start_year = int(row['Учебный год'].split(' - ')[0])
    course = (start_year % 100) - group_year + 1
    semester = course * 2 - 1 if row['Полугодие'] == 1 else course * 2
    return semester

df_new['Семестр'] = df_new.apply(calculate_semester, axis=1)

In [77]:
df_new['Программа'] = df_new['Учебная группа'].str.split('-').str[0]

In [78]:
df_new.head(2)

,hash,Номер ЛД,Уровень подготовки,Учебная группа,Специальность/направление,Учебный год,Полугодие,Дисциплина,Оценка (без пересдач),Оценка (успеваемость),Семестр,Программа
0,67a80fffd8d0294a596eda117d7e393c,2010218,Академический бакалавр,БИВТ-20-4,Информатика и вычислительная техника,2020 - 2021,1,Инженерная компьютерная графика,Хорошо,Хорошо,1,БИВТ
1,67a80fffd8d0294a596eda117d7e393c,2010218,Академический бакалавр,БИВТ-20-4,Информатика и вычислительная техника,2020 - 2021,1,Иностранный язык,Удовлетворительно,Удовлетворительно,1,БИВТ


In [79]:
disciplines = len(df_new['Дисциплина'].unique())
print(f'Количество уникальных дисциплин: {disciplines}')

Количество уникальных дисциплин: 658


In [80]:
programs = len(df_new['Программа'].unique())
print(f'Количество уникальных программ: {programs}')

Количество уникальных программ: 19


In [81]:
df = df_new
df = df.drop(columns=['Номер ЛД', 'Учебная группа', 'Уровень подготовки', 'Учебный год', 'Полугодие', 'Специальность/направление'])

In [82]:
df.head(1)

,hash,Дисциплина,Оценка (без пересдач),Оценка (успеваемость),Семестр,Программа
0,67a80fffd8d0294a596eda117d7e393c,Инженерная компьютерная графика,Хорошо,Хорошо,1,БИВТ


#### Кодирование оценок

In [83]:
df.replace({'зачтено': 5, 'Отлично': 5,
            'Хорошо': 4,
            'Удовлетворительно': 3,
            'Неудовлетворительно': 2, 'Неявка': 2, 'не зачтено': 2, 'Не допущен': 2,
            'Неявка по ув.причине': 0
            },
           inplace=True)

Так как по ТЗ все оставшиеся null у студента это 2, то заменим их:

In [84]:
df.fillna(2, inplace=True)

In [85]:
df.head(2)

,hash,Дисциплина,Оценка (без пересдач),Оценка (успеваемость),Семестр,Программа
0,67a80fffd8d0294a596eda117d7e393c,Инженерная компьютерная графика,4,4.0,1,БИВТ
1,67a80fffd8d0294a596eda117d7e393c,Иностранный язык,3,3.0,1,БИВТ


#### Создание столбцов для каждого направления

In [86]:
df_programs = pd.get_dummies(df, columns=['Программа'], prefix='', prefix_sep='')
df_programs = df_programs.drop(columns=['Дисциплина', 'Оценка (без пересдач)', 'Оценка (успеваемость)'])
df_programs = df_programs.astype({col: int for col in df_programs.columns[1:]})
df_programs = df_programs.drop_duplicates()

In [87]:
df_programs.head()

,hash,Семестр,ББИ,БИВТ,БИСТ,БЛГ,БМН,БМТ,БМТМ,БНМ,...,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
0,67a80fffd8d0294a596eda117d7e393c,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,67a80fffd8d0294a596eda117d7e393c,2,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,67a80fffd8d0294a596eda117d7e393c,3,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,67a80fffd8d0294a596eda117d7e393c,4,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33,67a80fffd8d0294a596eda117d7e393c,5,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Создание столбцов для каждой дисциплины

Сопоставление промежуточной оценки соответствующей дисциплине

In [88]:
pivot_df = df.pivot_table(index=['hash', 'Семестр'],
                          columns='Дисциплина',
                          values='Оценка (без пересдач)',
                          aggfunc='first'
                          ).reset_index()

pivot_df.columns.name = None
pivot_df.columns = [str(col) for col in pivot_df.columns]

Заполнение отсутствующих предметов нулями

In [89]:
pivot_df.fillna(0, inplace=True)

In [90]:
pivot_df = pivot_df.astype({col: int for col in pivot_df.columns[1:]})

In [91]:
pivot_df.head(2)

,hash,Семестр,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,Python для извлечения и обработки данных,...,Электроника и измерительная техника,Электроснабжение горных предприятий,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика
0,000006af6e40c8234a5af27896b7bba5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,000006af6e40c8234a5af27896b7bba5,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Подсчёт количества двоек в каждом семестре

In [92]:
df = pivot_df
df.head(7)

,hash,Семестр,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,Python для извлечения и обработки данных,...,Электроника и измерительная техника,Электроснабжение горных предприятий,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика
0,000006af6e40c8234a5af27896b7bba5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,000006af6e40c8234a5af27896b7bba5,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,000006af6e40c8234a5af27896b7bba5,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,000006af6e40c8234a5af27896b7bba5,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,000006af6e40c8234a5af27896b7bba5,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,000006af6e40c8234a5af27896b7bba5,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,00083ac3c8aecc3a1cf66029173e56fa,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
grades_columns = df.drop(['hash', 'Семестр'], axis=1)

In [94]:
df['count_2'] = (df == 2).sum(axis=1)
df.head(7)

<ipython-input-94-98367a2bc522>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['count_2'] = (df == 2).sum(axis=1)


,hash,Семестр,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,Python для извлечения и обработки данных,...,Электроснабжение горных предприятий,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика,count_2
0,000006af6e40c8234a5af27896b7bba5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,000006af6e40c8234a5af27896b7bba5,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2,000006af6e40c8234a5af27896b7bba5,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,000006af6e40c8234a5af27896b7bba5,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
4,000006af6e40c8234a5af27896b7bba5,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
5,000006af6e40c8234a5af27896b7bba5,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
6,00083ac3c8aecc3a1cf66029173e56fa,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [95]:
# Смещение количества двоек наверх на одну строку для каждого студента
df['next_sem_debts'] = df['count_2'].shift(-1).copy()
# Если встретился следующий студент - последнему известному семестру текущего
# студента соответсвует данное количество долгов без смещения
df['next_sem_debts'] = np.where(df['hash'] != df['hash'].shift(-1), df['count_2'], df['next_sem_debts'])
df['next_sem_debts'].fillna(df['count_2'], inplace=True)
df = df.drop('count_2', axis=1)

<ipython-input-95-a9c68e21a63d>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['next_sem_debts'] = df['count_2'].shift(-1).copy()


In [96]:
df['next_sem_debts'] = df['next_sem_debts'].astype(int)

In [97]:
df

,hash,Семестр,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,Python для извлечения и обработки данных,...,Электроснабжение горных предприятий,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика,next_sem_debts
0,000006af6e40c8234a5af27896b7bba5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1,000006af6e40c8234a5af27896b7bba5,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2,000006af6e40c8234a5af27896b7bba5,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,000006af6e40c8234a5af27896b7bba5,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
4,000006af6e40c8234a5af27896b7bba5,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18094,ffed4fe2ad8cbff97e0b3d21da23008d,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
18095,ffed4fe2ad8cbff97e0b3d21da23008d,5,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,7
18096,ffed4fe2ad8cbff97e0b3d21da23008d,6,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
18097,ffed4fe2ad8cbff97e0b3d21da23008d,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


#### Подсчёт количества каждого вида оценок

для экспорта в другую модель

In [98]:
subjects = df.apply(pd.Series.value_counts, axis=1)[[2, 3, 4, 5]].fillna(0)
subjects = subjects.astype(int)

#subjects['total'] = subjects[[2, 3, 4, 5]].sum(axis=1)
#subjects['total'] = pd.to_numeric(subjects['total'], errors='coerce').fillna(0)
subjects.head()

,2,3,4,5
0,2,2,1,3
1,4,3,2,2
2,4,2,2,2
3,4,3,1,2
4,7,1,1,1


In [99]:
df.head()

,hash,Семестр,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,Python для извлечения и обработки данных,...,Электроснабжение горных предприятий,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика,next_sem_debts
0,000006af6e40c8234a5af27896b7bba5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1,000006af6e40c8234a5af27896b7bba5,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2,000006af6e40c8234a5af27896b7bba5,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,000006af6e40c8234a5af27896b7bba5,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
4,000006af6e40c8234a5af27896b7bba5,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9


#### Смещение столбцов

In [100]:
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
print(cols)

['next_sem_debts', 'hash', 'Семестр', 'BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений', 'BIM-технологии при проектировании горнодобывающих комплексов', 'CAD системы в горном производстве', 'CAD системы в подземном строительстве', 'Cертификация в горном деле', 'HR-системы организационного управления предприятием', 'Linux', 'Python для извлечения и обработки данных', 'Автоматизация горных машин и установок', 'Автоматизация письменного перевода и управление терминологией', 'Автоматизация технологических процессов', 'Автоматизированное проектирование машин', 'Автоматизированный электропривод машин и установок', 'Автомобили и тракторы', 'Академический английский, часть 1', 'Академический английский, часть 2', 'Академический английский. Часть 1', 'Алгоритмы анализа данных', 'Алгоритмы дискретной математики', 'Алгоритмы и структуры данных', 'Анализ данных и аналитика в принятии решений', 'Анализ данных на практике', 'Анализ и моделирование бизнес-процессов'

In [101]:
df = df[cols]

In [102]:
lessons = df.columns.to_list()[2:]
print(lessons)

['Семестр', 'BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений', 'BIM-технологии при проектировании горнодобывающих комплексов', 'CAD системы в горном производстве', 'CAD системы в подземном строительстве', 'Cертификация в горном деле', 'HR-системы организационного управления предприятием', 'Linux', 'Python для извлечения и обработки данных', 'Автоматизация горных машин и установок', 'Автоматизация письменного перевода и управление терминологией', 'Автоматизация технологических процессов', 'Автоматизированное проектирование машин', 'Автоматизированный электропривод машин и установок', 'Автомобили и тракторы', 'Академический английский, часть 1', 'Академический английский, часть 2', 'Академический английский. Часть 1', 'Алгоритмы анализа данных', 'Алгоритмы дискретной математики', 'Алгоритмы и структуры данных', 'Анализ данных и аналитика в принятии решений', 'Анализ данных на практике', 'Анализ и моделирование бизнес-процессов', 'Анализ точности маркшей

In [103]:
df.head(3)

,next_sem_debts,hash,Семестр,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,...,Электроника и измерительная техника,Электроснабжение горных предприятий,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика
0,4,000006af6e40c8234a5af27896b7bba5,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,000006af6e40c8234a5af27896b7bba5,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,000006af6e40c8234a5af27896b7bba5,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
df = df.astype({col: int for col in df.columns[2:]})

In [105]:
df.head(3)

,next_sem_debts,hash,Семестр,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,...,Электроника и измерительная техника,Электроснабжение горных предприятий,Электротехника,Электротехника и электроника,Электротехническое и конструкционное материаловедение,Электротехнологические установки,Элементы систем автоматического управления,Энергетика горных предприятий,Энергоемкость технологических процессов,Эргономика
0,4,000006af6e40c8234a5af27896b7bba5,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,000006af6e40c8234a5af27896b7bba5,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,000006af6e40c8234a5af27896b7bba5,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Объединение таблицы оценок с таблицей направлений

In [106]:
df_programs.head(1)

,hash,Семестр,ББИ,БИВТ,БИСТ,БЛГ,БМН,БМТ,БМТМ,БНМ,...,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
0,67a80fffd8d0294a596eda117d7e393c,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
df_merged = pd.merge(df, df_programs, on=['hash', 'Семестр'], how='inner')

In [108]:
df_merged.head()

,next_sem_debts,hash,Семестр,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,...,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
0,4,000006af6e40c8234a5af27896b7bba5,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,000006af6e40c8234a5af27896b7bba5,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,000006af6e40c8234a5af27896b7bba5,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7,000006af6e40c8234a5af27896b7bba5,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,000006af6e40c8234a5af27896b7bba5,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Обработка данных для другой модели, не учитывающей дисциплины

In [109]:
column_list = df_merged.columns.to_list()
column_list = column_list[:3] + column_list[-19:]
print(column_list)

['next_sem_debts', 'hash', 'Семестр', 'ББИ', 'БИВТ', 'БИСТ', 'БЛГ', 'БМН', 'БМТ', 'БМТМ', 'БНМ', 'БНМТ', 'БПИ', 'БПМ', 'БТМО', 'БФЗ', 'БЭК', 'БЭН', 'БЭЭ', 'СГД', 'СНТС', 'СФП']


In [110]:
df_merged_cnt = df_merged[column_list]
df_merged_cnt = subjects.join(df_merged_cnt, how='inner')
df_merged_cnt.head(2)

,2,3,4,5,next_sem_debts,hash,Семестр,ББИ,БИВТ,БИСТ,...,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
0,2,2,1,3,4,000006af6e40c8234a5af27896b7bba5,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,3,2,2,4,000006af6e40c8234a5af27896b7bba5,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [111]:
df_merged_cnt.to_csv('num_of_debts.csv', index=False)

## Загрузка данных в модель

In [112]:
results = df_merged['next_sem_debts'].to_numpy()
print(f'Минимальное количество двоек: {results.min()}')
print(f'Максимальное количество двоек: {results.max()}')
print(f'Среднее количество двоек: {results.mean():.3f}')

Минимальное количество двоек: 0
Максимальное количество двоек: 15
Среднее количество двоек: 2.226


In [113]:
df_merged = df_merged.drop('hash', axis=1)

In [114]:
df_merged.head(2)

,next_sem_debts,Семестр,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,Python для извлечения и обработки данных,...,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
0,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Разделение на трейн-тест

In [115]:
X = df_merged.drop('next_sem_debts', axis=1)
X = X.round().astype(int)
y = df_merged['next_sem_debts']
y = y.astype(int)

In [116]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [117]:
X_train

,Семестр,"BIM-технологии в проектировании, строительстве и эксплуатации подземных сооружений",BIM-технологии при проектировании горнодобывающих комплексов,CAD системы в горном производстве,CAD системы в подземном строительстве,Cертификация в горном деле,HR-системы организационного управления предприятием,Linux,Python для извлечения и обработки данных,Автоматизация горных машин и установок,...,БПИ,БПМ,БТМО,БФЗ,БЭК,БЭН,БЭЭ,СГД,СНТС,СФП
15871,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
12656,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1521,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5083,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5456,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
11964,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5390,6,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
860,6,0,0,0,5,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [118]:
y_train

15871    2
12656    7
1521     8
5083     0
5456     4
        ..
11284    0
11964    0
5390     2
860      1
15795    0
Name: next_sem_debts, Length: 13578, dtype: int64

In [119]:
train_pool = Pool(X_train,
                  label=y_train)

test_pool = Pool(X_test,
                 label=y_test)

#### Подбор гиперпараметров

In [120]:
import optuna
from sklearn.model_selection import cross_val_score


def objective(trial):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'random_seed': 42,
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'loss_function': 'RMSE',
        'eval_metric': 'R2',
    }
    model = CatBoostRegressor(**params, verbose=False)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=20, verbose=False)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print('Best trial:')
best_trial = study.best_trial
print(f'  Value: {best_trial.value:.4f}')
print('  Params: ')
for key, value in best_trial.params.items():
    print(f'    {key}: {value}')

ModuleNotFoundError: No module named 'optuna'

In [121]:
model = CatBoostRegressor(custom_metric=['R2'], iterations=1000, learning_rate=0.2)
model.fit(train_pool, eval_set=[test_pool], verbose=200)

0:	learn: 2.3265250	test: 2.3212141	best: 2.3212141 (0)	total: 29.7ms	remaining: 29.7s
200:	learn: 1.3767997	test: 1.4913445	best: 1.4913445 (200)	total: 1.47s	remaining: 5.85s
400:	learn: 1.2521089	test: 1.4529422	best: 1.4529422 (400)	total: 2.57s	remaining: 3.84s
600:	learn: 1.1851390	test: 1.4455888	best: 1.4455495 (599)	total: 3.68s	remaining: 2.45s
800:	learn: 1.1373693	test: 1.4426670	best: 1.4424290 (798)	total: 4.79s	remaining: 1.19s
999:	learn: 1.1006451	test: 1.4427240	best: 1.4412444 (960)	total: 5.88s	remaining: 0us

bestTest = 1.441244374
bestIteration = 960

Shrink model to first 961 iterations.


In [122]:
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MAE: {mae:.4f}')
print(f'R2: {r2:.4f}')

MAE: 0.9538
R2: 0.6663


## Класс студента

In [123]:
import json
with open('lessons_next.json', 'w', encoding='utf-8') as file:
  json.dump(lessons, file, ensure_ascii=False)

In [124]:
lessons = lessons
# в приложении все дисциплины загружаются из json файла
programs = df_programs.columns.to_list()[2:]

class NewStudent:
  def __init__(self):
        self.lessons = lessons
        self.programs = programs
        self.data = {key: np.NaN for key in list(self.lessons) + list(self.programs)}


  def add_score(self, key, score):
    if key in self.data:
      self.data[key] = score


  def load_data_from_dict(self, data_dict):
      for lesson, score in data_dict.items():
          if lesson in self.data:
              self.data[lesson] = score

  def data_processing(self):
    student_data = np.array(self.prepare_data_for_prediction()).reshape(1, -1)
    student_data = np.nan_to_num(student_data, nan=0, copy=True)
    student_data = student_data.astype(int)
    return student_data

  def prepare_data_for_prediction(self):
    return [self.data[key] for key in self.data]

In [125]:
student = NewStudent()
student.add_score('БПМ', 1)
student.add_score('Семестр', 1)
student.add_score('Введение в специальность', 5)
student.add_score('Вычислительные машины, сети и системы', 5)
student.add_score('Иностранный язык', 5)
student.add_score('Математика', 5)
student.add_score('Программирование и алгоритмизация', 5)
student.add_score('Физика', 5)
student.add_score('Физическая культура и спорт', 5)
student.add_score('Объектно-ориентированное программирование', 5)

Предскажем долю двоек в следующем семестре

In [126]:
res = model.predict(student.data_processing()).round().astype(int)
print(f'Количество двоек в следующем семестре: {res[0]}')

Количество двоек в следующем семестре: 1


## Экспорт модели

In [127]:
import pickle

with open('scoring_next_sem.pkl', 'wb') as file:
  pickle.dump(model, file)